In [1]:
import import_ipynb
import CA
import pprint
CAPubKeyC = []
reqFrame = {}
clientName = 'ECU1'
role = 4
ClientCert = []
clientPubKey = []
clientPriKey = []

importing Jupyter notebook from CA.ipynb


In [2]:
def CertificateRoutine():
    global ClientCert, clientPubKey, clientPriKey, CAPubKeyC, reqFrame, clientName, role
    import socket
    import pickle
    client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client.connect(('127.0.0.7', 8080))
    global ClientCert
    if CAPubKeyC == []:
        reqFrame = {"req":1} # requesting for CA's PublicKey
        data = pickle.dumps(reqFrame)
        print("Requesting for CA's PublicKey \t:\t")
        pprint.pprint(data)
        print('\n')
        client.send(data)
        data = client.recv(4096)
        CAPubKeyC = pickle.loads(data)
        print("CA's PublicKey \t:\t")
        pprint.pprint(data)
        print('\n')
    if ClientCert == []: #Generating Subject'S Public,Private Key pair
        if clientPubKey == []:
            clientPubKey , clientPriKey = CA.KeyGen()
        reqFrame = {"req" : 2, "subName" : clientName, "role":role, "pubKey" : clientPubKey} #Request for certificate
        data = pickle.dumps(reqFrame)
        print("Request for Certificate Generation \t:\t")
        pprint.pprint(data)
        print('\n')
        client.send(data)
        data = client.recv(4096)
        fromServer = pickle.loads(data)
#         print(fromServer)
        ClientCert = CA.certificate()
        ClientCert.dict2cert(fromServer)
        print("Generated Certificate \t:\t")
        pprint.pprint(ClientCert.transmitCertificate())
        print('\n')
    client.close()

In [3]:
import numpy as np

def RandomGenerator():
    random_bytes = str(np.random.bytes(10).hex())
    seed = random_bytes
#     for i in range(0,2):
#         random_bytes = str(np.random.bytes(1).hex())
#         seed = seed +' '+ random_bytes
    return seed.upper()

def Req(data):
    global ClientCert, clientPriKey
    if data["sFn"] == '08':
        print("From Tester : ")
        pprint.pprint(data)
        print("\n")
        from_client = {}
        from_client["Fn"] = '69'
        from_client["sFn"] = '08'
        from_client["Cnf"] = '02'
        print("To Tester : ")
        pprint.pprint(from_client)
        print("\n")
        return from_client
    elif data["sFn"] == '01' or data['sFn']=='02':
        print("--------------Unidirectional Authetication")
        print("\n\n")
        print("From Tester : ")
        pprint.pprint(data)
        print("\n")
        from_client = {}
        from_client["Fn"] = '69'
        from_client["sFn"] = data['sFn']
        from_client["Cnf"] = '11'
        t = RandomGenerator()
        from_client["CS"] = t
        if from_client["sFn"] == '02':
            temp_ccli = data["CCli"]
            from_client['ECer'] = ClientCert
            from_client['POOC'] = CA.sign(temp_ccli,clientPriKey)
        print("To Tester : ")
        pprint.pprint(from_client)
        print("\n")
        return from_client
    elif data["sFn"] == '03':
        print("From Tester : ")
        pprint.pprint(data)
        print("\n")
        from_client = {}
        from_client["Fn"] = '69'
        from_client["sFn"] = '03'
        from_client["Cnf"] = '12'
        print("To Tester : ")
        pprint.pprint(from_client)
        print("\n")
        return from_client
    return 0

In [4]:

def AuthenticationRequester():
    import socket
    import pickle
    serv = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    serv.bind(("127.0.0.1", 9090))
    serv.listen(5)
    a,b=0,0
    global RecCer, CAPubKeyC, ClientCert , clientPubKey, clientPriKey
    while a==0:
        b=0
        conn, addr = serv.accept()
        while b == 0:
            data = conn.recv(4096)
            if not data: break
            from_client = pickle.loads(data)
            if from_client["sFn"] == '08':
                tempData = Req(from_client)
                from_client = tempData
                data = pickle.dumps(from_client)
                conn.send(data)
                data = conn.recv(4096)
                from_client = pickle.loads(data)
            if from_client["sFn"] == '01' or from_client["sFn"] == '02':
                RecCert = from_client['Cert']
                if CA.verifyCertificate(RecCert,CAPubKeyC ):
                    print("--------------Certificate Verified\n")
                    tempData = Req(from_client)
                    from_client = tempData

                    t = from_client['CS']
                    data = pickle.dumps(from_client)
                    conn.send(data)
                    data = conn.recv(4096)
                    from_client = pickle.loads(data)
                    if from_client["sFn"] == '03':
                        if CA.verifySign(t,from_client["POO"],RecCert.subjectPublicKey):
                            print("--------------Certificate Verified with proof of Ownership\n")
                            tempData = Req(from_client)
                            from_client = tempData
                            data = pickle.dumps(from_client)
                            conn.send(data)
                            conn.close()
            conn.close()
            a,b=1,1

In [5]:
CertificateRoutine()

Requesting for CA's PublicKey 	:	
b'\x80\x04\x95\x0c\x00\x00\x00\x00\x00\x00\x00}\x94\x8c\x03req\x94K\x01s.'


CA's PublicKey 	:	
(b'\x80\x04\x95g\x00\x00\x00\x00\x00\x00\x00\x8c\x07rsa.key\x94\x8c\tPublicKey'
 b'\x94\x93\x94)\x81\x94\x8aA\xa7\x8dO\xb6\xdb\xd8\x18\xa8F{m\x9b\x80\x1c\xb4/'
 b'\x0e\xa6\xfe\xd7:\xe3\xb1\xa6*T\xff\xbe\xac8\xfbi0\x188p3\x84\xa7\xe3'
 b'B\x9a2\xaf\x17\xce\xddc"\x01\xfe\x81\x8cF?\xb7\xc4,>b\x0bo8\x9b\x00J\x01\x00'
 b'\x01\x00\x86\x94b.')


Request for Certificate Generation 	:	
(b'\x80\x04\x95\x96\x00\x00\x00\x00\x00\x00\x00}\x94(\x8c\x03req\x94'
 b'K\x02\x8c\x07subName\x94\x8c\x04ECU1\x94\x8c\x04role\x94K\x04\x8c\x06pubKey'
 b'\x94\x8c\x07rsa.key\x94\x8c\tPublicKey\x94\x93\x94)\x81\x94\x8aA\xd1y'
 b'\xb55\x983\x89\xa5\xad;\xe0>L\x1d\xf0\xb3\xbd\x83.\xfe\xba\x84\xb2{\x8fi'
 b'\x9b\xb7\x89\xab\tMp6XI&\x16\xd0\xed\x96\x94\xfczqc\x0bfp+\x91\x8ciK'
 b'\xa4\x17U\xa8\x07\xbb\xc1\x07*\xc5\x00J\x01\x00\x01\x00\x86\x94bu.')


Generated Certificate 	:	
{'ext': 4,
 'i_n

In [7]:
AuthenticationRequester()

From Tester : 
{'Fn': '29', 'sFn': '08'}


To Tester : 
{'Cnf': '02', 'Fn': '69', 'sFn': '08'}


--------------Verification succeed

--------------Certificate Verified

--------------Unidirectional Authetication



From Tester : 
{'CC': '00',
 'CCli': '8DDEDE24863B97A6EE3D',
 'Cert': <CA.certificate object at 0x0000019840AEE1C0>,
 'Fn': '29',
 'sFn': '02'}


To Tester : 
{'CS': '96526251547CE99B9D81',
 'Cnf': '11',
 'ECer': <CA.certificate object at 0x00000198314D5D00>,
 'Fn': '69',
 'POOC': b'\xaf\x8e\xcb\xc5\xcc\x11\xb2dY\xe0\x02f`\x0e3\xdf`\x95\xb9$'
         b']\xd2\xb9\x91\xaf0\xb8\x85\xe5G}\xb0\xa7\xe1\xe7\xfa\x0ef\xb1\x87'
         b"]\x9a\x9c\x91\xd9\xed\x87\xe39>\xaeGdD\x8c\xd3F\xa8\x01\x1f/'\xd1_",
 'sFn': '02'}


--------------Verification succeed

--------------Certificate Verified with proof of Ownership

From Tester : 
{'Fn': '29',
 'POO': b'&:)\xc0\xc3\xf8\x085\xae\x87B\x00\xc3\xd7\xc4\xe7\xee\xfb\x8e\x80'
        b'CG\xac1\xcao\x1d\x9e\x8e\x87\xd0\xc6Q\xf3l8\xb4f:\x12\x

In [28]:
AuthenticationRequester()

From Tester : 
{'Fn': '29', 'sFn': '08'}


To Tester : 
{'Cnf': '02', 'Fn': '69', 'sFn': '08'}


Verification succeed
--------------Certificate Verified

--------------Unidirectional Authetication



From Tester : 
{'CC': '00',
 'CCli': '4AC194C635569536BB00',
 'Cert': <CA.certificate object at 0x000001918CC192E0>,
 'Fn': '29',
 'sFn': '02'}


To Tester : 
{'CS': 'EA9DB1CCE316ECA89047',
 'Cnf': '11',
 'ECer': <CA.certificate object at 0x000001918CBD5A90>,
 'Fn': '69',
 'POOC': b'\x87s\x8e\x0b(\x12\x88\x9c\x1d\xbf\xca^>w\x0f\xef\xeb\x05\x8c\x13'
         b'\x91\xf7`HC\t5V\xe4\\\xfeY\x93"\x9f\n\x01\xd6]\x17\x00\xca\xbd^'
         b'\xd9\xef\xdf\x89u\xe7\x13\xbc\xd3\xc2\x92U\x12cL\xc7\x18\xa0\xdd\xef',
 'sFn': '02'}


Verification succeed
--------------Certificate Verified with proof of Ownership

From Tester : 
{'Fn': '29',
 'POO': b'\x0e\xdf\xce<?\xdb+L\xf1\xa8\x18_wc\xc7K\xa6\xc4\x7f\xa0+\x08x{'
        b'18\x8f\xa1\x05\xeeo\xd9\xdaw\xa6sM\x19xj\xbeN\x87\xc1\xfa9\xa9\x0c'
        b'\xf

In [13]:
# CA.verifyCertificate(ClientCert,CAPubKeyC )

# str(type(ClientCert))

# print(CA.CAPubKey)

# print(CAPubKeyC)

In [16]:
# t= RandomGenerator()
# txt = CA.sign(t,clientPriKey)
# print(CA.verifySign(t,txt,clientPubKey))
# conn.close()

In [ ]:
# # Hardcoded to inputfn with bidirectional authentication


# import numpy as np
# def RandomGenerator():
#     random_bytes = str(np.random.bytes(10).hex())
#     seed = random_bytes
# #     for i in range(0,2):
# #         random_bytes = str(np.random.bytes(1).hex())
# #         seed = seed +' '+ random_bytes
#     return seed.upper()
# def AuthenticationRequester():
#     import socket
#     import pickle
#     serv = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
#     serv.bind(("127.0.0.1", 9090))
#     serv.listen(5)
#     a,b=0,0
#     global RecCer, CAPubKeyC, ClientCert , clientPubKey, clientPriKey
#     while a==0:
#         b=0
#         conn, addr = serv.accept()
#         while b == 0:
#             data = conn.recv(4096)
#             if not data: break
#             from_client = pickle.loads(data)
#             print("From Tester : ")
#             pprint.pprint(from_client)
#             print("\n")
#             if from_client["sFn"] == '08':
#                 tempData = Req(from_client)
#                 from_client = tempData
# #                 from_client["Fn"] = '69'
# #                 from_client["sFn"] = '08'
# #                 from_client["Cnf"] = '02'
#                 print("To Tester : ")
#                 pprint.pprint(from_client)
#                 print("\n")
#                 data = pickle.dumps(from_client)
#                 conn.send(data)
#                 data = conn.recv(4096)
#                 from_client = pickle.loads(data)
#                 print("From Tester : ")
#                 pprint.pprint(from_client)
#                 print("\n")
#             if from_client["sFn"] == '01' or from_client["sFn"] == '02':
# #                 temp_1 = from_client["sFn"]
# #                 if from_client['CC'] == '01':
# #                     genSessionKey =1 
#                 RecCert = from_client['Cert']
#                 if CA.verifyCertificate(RecCert,CAPubKeyC ):
#                     print("Certificate Verified")
# #                     temp_ccli = from_client["CCli"]
# #                     from_client = {}
# #                     from_client["Fn"] = '69'
# #                     from_client["sFn"] = temp_1
# #                     from_client["Cnf"] = '11'
# #                     t = RandomGenerator()
# #                     from_client["CS"] = t
# #                     if from_client["sFn"] == '02':
# #                         from_client['ECer'] = ClientCert
# #                         from_client['POOC'] = CA.sign(temp_ccli,clientPriKey)
#                     tempData = Req(from_client)
#                     from_client = tempData
#                     print("To Tester : ")
#                     pprint.pprint(from_client)
#                     print("\n")
#                     t = from_client['CS']
#                     data = pickle.dumps(from_client)
#                     conn.send(data)
#                     data = conn.recv(4096)
#                     from_client = pickle.loads(data)
#                     print("From Tester : ")
#                     pprint.pprint(from_client)
#                     print("\n")
#                     if from_client["sFn"] == '03':
#                         if CA.verifySign(t,from_client["POO"],RecCert.subjectPublicKey):
#                             print("Certificate Verified with proof of Ownership")
#                             tempData = Req(from_client)
#                             from_client = tempData
# #                             from_client = {}
# #                             from_client["Fn"] = '69'
# #                             from_client["sFn"] = '03'
# #                             from_client["Cnf"] = '12'
#                             print("To Tester : ")
#                             pprint.pprint(from_client)
#                             print("\n")
#                             data = pickle.dumps(from_client)
#                             conn.send(data)
#                             conn.close()
#             conn.close()
#             a,b=1,1

In [4]:
## Hardcoded without bidirectional authentication


# import numpy as np
# def RandomGenerator():
#     random_bytes = str(np.random.bytes(10).hex())
#     seed = random_bytes
# #     for i in range(0,2):
# #         random_bytes = str(np.random.bytes(1).hex())
# #         seed = seed +' '+ random_bytes
#     return seed.upper()
# def AuthenticationRequester():
#     import socket
#     import pickle
#     serv = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
#     serv.bind(("127.0.0.1", 9090))
#     serv.listen(5)
#     a,b=0,0
#     global RecCer, CAPubKeyC
#     while a==0:
#         b=0
#         conn, addr = serv.accept()
#         while b == 0:
#             data = conn.recv(4096)
#             if not data: break
#             from_client = pickle.loads(data)
#             print("From Tester : ")
#             pprint.pprint(from_client)
#             print("\n")
#             if from_client["sFn"] == '08':
#                 from_client = {}
#                 from_client["Fn"] = '69'
#                 from_client["sFn"] = '08'
#                 from_client["Cnf"] = '02'
#                 print("To Tester : ")
#                 pprint.pprint(from_client)
#                 print("\n")
#                 data = pickle.dumps(from_client)
#                 conn.send(data)
#                 data = conn.recv(4096)
#                 from_client = pickle.loads(data)
#                 print("From Tester : ")
#                 pprint.pprint(from_client)
#                 print("\n")
#             if from_client["sFn"] == '01':
# #                 if from_client['CC'] == '01':
# #                     genSessionKey =1
#                 RecCert = from_client['Cert']
#                 if CA.verifyCertificate(RecCert,CAPubKeyC ):
#                     print("Certificate Verified")
#                     from_client = {}
#                     from_client["Fn"] = '69'
#                     from_client["sFn"] = '01'
#                     from_client["Cnf"] = '11'
#                     t = RandomGenerator()
#                     from_client["CS"] = t
#                     print("To Tester : ")
#                     pprint.pprint(from_client)
#                     print("\n")
#                     data = pickle.dumps(from_client)
#                     conn.send(data)
#                     data = conn.recv(4096)
#                     from_client = pickle.loads(data)
#                     print("From Tester : ")
#                     pprint.pprint(from_client)
#                     print("\n")
#                     if from_client["sFn"] == '03':
#                         if CA.verifySign(t,from_client["POO"],RecCert.subjectPublicKey):
#                             print("Certificate Verified with proof of Ownership")
#                             from_client = {}
#                             from_client["Fn"] = '69'
#                             from_client["sFn"] = '03'
#                             from_client["Cnf"] = '12'
#                             print("To Tester : ")
#                             pprint.pprint(from_client)
#                             print("\n")
#                             data = pickle.dumps(from_client)
#                             conn.send(data)
#                             conn.close()
#             conn.close()
#             a,b=1,1

In [8]:
AuthenticationRequester()

From Tester : 
{'Fn': '29', 'sFn': '08'}


To Tester : 
{'Cnf': '02', 'Fn': '69', 'sFn': '08'}


From Tester : 
{'CC': '00',
 'Cert': <CA.certificate object at 0x00000187C62939D0>,
 'Fn': '29',
 'sFn': '01'}


Verification succeed
Certificate Verified
To Tester : 
{'CS': 'D3894C2D32D2F183036D', 'Cnf': '11', 'Fn': '69', 'sFn': '01'}


From Tester : 
{'Fn': '29',
 'POO': b'l1}\x98;\xce\xc9\xab\xfemy\xbe"P\xf0\xa8;^\x7f\x9b.\xce\xc7\x89'
        b'\x19\xa6%\xeaE\xa7\xa8\xafk\xd2\x9e9#\xb73\x0fs\xf0\x00\x88\\#&\x13'
        b'zl\xf5\x12u\xda\x1d6\xc6\x9f2Wj7\xfcj',
 'sFn': '03'}


Verification succeed
Certificate Verified
To Tester : 
{'Cnf': '12', 'Fn': '69', 'sFn': '03'}


